In [1]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from IPython.display import clear_output
import time
import os

pd.set_option('display.max_columns', None)


2024-02-06 16:24:07.940158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Initial Research

MediaPipe has a z-axis output, which apparently estimates the pose in a 2m3 box 
wherer the origin is the center of the hip. As usual, the point of the algo is 
to determine the _relataive_ position of the key points, not their absolute
position or distance. MP does provide this as `world_coordinates`


# Mediapipe

Simple framework:
1. Capture the frame from the video feed.
2. Process the frame using the pose detection model to find keypoints.
3. Draw the keypoints directly onto the frame.
4. Display the frame with the drawn keypoints. 

Below is some code using MediaPipe to get the key metrics out of the pose estimation.
**MODEL COMPLEXITY:** The intended moden (lite, full, heavy) can be adjusted my changing the `model_complexity` kwarg when initializing the `pose` variable.

In [12]:
'''
DEPRICATED, USING OLD VERSION WITHOUT VISIBILITY
'''

import cv2
import mediapipe as mp

# Initialize MediaPipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, model_complexity=2)
mp_drawing = mp.solutions.drawing_utils

# Start capturing video from the webcam.
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB.
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose.
    results = pose.process(frame_rgb)

    # Draw the pose annotations on the frame.
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the frame.
    cv2.imshow('Mediapipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the webcam and destroy all OpenCV windows.
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


I0000 00:00:1707185922.848916       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-22.1.29), renderer: Intel(R) Iris(TM) Plus Graphics 655
2024-02-06 11:18:43.418 python[30127:181416] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


-1

: 

Another implementation example, with some readouts. This is also mediapipe.

In [29]:
'''
DEPRICATED, USING OLD VERSION WITHOUT VISIBILITY
'''

# Initialize stuff from mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        try:
            landmarks = results.pose_landmarks.landmark
            print(f"R_Sh:\n{landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]}")
            print(f"L_Sh:\n{landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]}")

            # Save values if torso Z values close to zero
            if np.max(np.abs([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z,
                              landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z,
                              landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z])) > 0.5 and\
                np.min([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility]) > 0.9:
                print("Can See Torso")

                # TODO Write Framework here to save dataframe of correct torso values.
                # The Z value threshold in the above lines need tweaking as well.

        except:
            pass

        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Show detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2),)

        cv2.imshow("Mediapipe feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

        clear_output(wait=True)
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


I0000 00:00:1707144748.110895       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-22.1.29), renderer: Intel(R) Iris(TM) Plus Graphics 655


R_Sh:
x: 0.48348966
y: 0.78436434
z: -0.25170693
visibility: 0.9666917

L_Sh:
x: 0.62090105
y: 0.74045944
z: -0.13063358
visibility: 0.92903376



## Experimenting with asynchronous processing of frames

Another method to run Mediapipe, as described here:
https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python#live-stream

Results and performance of each models (lite, full, heavy) are summarized here: https://storage.googleapis.com/mediapipe-assets/Model%20Card%20BlazePose%20GHUM%203D.pdf

**Issues with asynchronous processing:** Looks like asynchronous readouts are not suitable if we are doing a video frame overlay. Keep synchronous processing such that overlay is still present with the live feed. The main code difference is using a timestamp in ms and also calling `detect_async()` instead of `detect()`.

**From the oracle**
> If you're running things synchronously, especially for real-time or live feed scenarios with frameworks like MediaPipe, you typically process each frame one at a time in a loop. In such cases, you don't necessarily need to use a callback function. Instead, you can directly process each frame as you capture it, analyze it with the pose detection model, and immediately draw the keypoints or landmarks onto the frame before displaying it. This approach ensures minimal delay between capturing a frame, processing it, and displaying the results, making it suitable for real-time applications.

In [3]:
"""
Belwo is a playaround with asynchronous processing, which is a bit more complex
than running a simple live feed. This method is better if we do not have any
overlay on the video, but ignore this part since it is not quite critical
to run thing asynchronously. Code is kept in notebook for reference.
"""

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import time

# Load model
model_path = "/Users/homemasaki/code/projects/fit_me/models/pose_landmarker_full.task"

# Drawing utility
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Create the task
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with the live stream mode:

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM)

# Grab frame using OpenCV
cap = cv2.VideoCapture(0)

with PoseLandmarker.create_from_options(options) as landmarker:
    while cap.isOpened():

        # Grab frame using OpenCV. Assume frame is grabbed.
        _, frame = cap.read()

        # Convert frame to MediaPipe Image object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame.tobytes(),
                            width=frame.shape[1], height=frame.shape[0])
        landmarker.detect(mp_image)

        #cv2.imshow("Mediapipe feed", frame)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


ValueError: The vision task is in live stream mode, a user-defined result callback must be provided.

## YOLO/Ultralytics

YOLO v8 has a pose estimation, but does not include many key ponts (17).
Also, YOLO is more known for object detection, hence may not be the best for this application.

Table this model for now, can return to it if we have reason.

# Extracting poses from still images for NN model

Below is code which takes in a directory, and can output a df including all KP and extracted data.

In [29]:
'''
DEPRICATED, USING OLD VERSION WITHOUT VISIBILITY
'''

import cv2
import mediapipe as mp
import pandas as pd
import glob

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1,
                    min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Placeholder for collected data
data = []

# Specify the directory path containing images
directory_path = 'new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_'

# Use glob to get all the image file paths
image_files = glob.glob(f'{directory_path}/*.jpg')

# Process each image
for image_file in image_files:
    image = cv2.imread(image_file)
    # Ensure the image was correctly loaded before proceeding
    if image is not None:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # Initialize a row with the image file name
        row = [image_file]

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Flatten all landmark data into the row
            for landmark in landmarks:
                row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        else:
            # If no landmarks are detected, fill in with None or a placeholder value
            row.extend([None] * 33 * 4)

        # Append the row to the data list
        data.append(row)
    else:
        print(f"Failed to load image: {image_file}")

# Define column names
columns = ['image_file']
for idx in range(1, 34):  # MediaPipe Pose has 33 landmarks, starting index at 1 for readability
    prefix = f'kp{idx}_'
    attributes = ['x', 'y', 'z', 'visibility']
    columns += [f'{prefix}{attr}' for attr in attributes]

# Create DataFrame
df = pd.DataFrame(data, columns=columns)


I0000 00:00:1707193516.069692       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-22.1.29), renderer: Intel(R) Iris(TM) Plus Graphics 655


Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__1_0_493.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__2_499.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__2_59.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__1_0_149.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__1_0_639.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__1_624.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pose_or_Virabhadrasana_I_/Warrior_I_Pose_or_Virabhadrasana_I__1_330.jpg
Failed to load image: new_pose_data/Yoga-82/images/Warrior_I_Pos

`pose_landmarker_result.pose_landmarks` gives normalized coordinates in normalized coordinates. Midpoint is the point between the hips.
`pose_landmarker_result.pose_world_landmarks` gives the coordinates (not normalized)

In [85]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import glob

model_path = "model_creator/pose_landmarker_full.task"

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

# Placeholder for collected data
data = []
attrs_to_get = ["x", "y", "z", "visibility", "presence"]
no_kp_files = []
fail_to_load_files = []

# Specify the directory path containing images
directory_path = 'selected_poses/akarna'

# Use glob to get all the image file paths
image_files = glob.glob(f'{directory_path}/*.jpg')

# Load the input image from an image file.
# mp_image = mp.Image.create_from_file('selected_poses/akarna/Screenshot 2024-02-06 at 12.50.57.png')

with PoseLandmarker.create_from_options(options) as landmarker:
    for image_file in image_files:
        # detect key points. object is pose_landmarker_result.pose_landmarks
        try:
            mp_image = mp.Image.create_from_file(image_file)
            pose_landmarker_result = landmarker.detect(mp_image)
            if len(pose_landmarker_result.pose_landmarks) > 0:
            # extract info out of landmarks object
                row = []
                for lmk in attrs_to_get:
                    for x in range(len(pose_landmarker_result.pose_landmarks[0])):
                        row.append(getattr(pose_landmarker_result.pose_landmarks[0][x], lmk))
                data.append(row)
            else:
                no_kp_files.append(image_file)
        except:
            fail_to_load_files.append(image_file)
            pass

columns = []
for x in range(33):
    for attr in attrs_to_get:
        col = f"kp{x}_{attr}"
        columns.append(col)

df = pd.DataFrame(data, columns=columns)
df


I0000 00:00:1707231003.013337       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-22.1.29), renderer: Intel(R) Iris(TM) Plus Graphics 655


,kp0_x,kp0_y,kp0_z,kp0_visibility,kp0_presence,kp1_x,kp1_y,kp1_z,kp1_visibility,kp1_presence,kp2_x,kp2_y,kp2_z,kp2_visibility,kp2_presence,kp3_x,kp3_y,kp3_z,kp3_visibility,kp3_presence,kp4_x,kp4_y,kp4_z,kp4_visibility,kp4_presence,kp5_x,kp5_y,kp5_z,kp5_visibility,kp5_presence,kp6_x,kp6_y,kp6_z,kp6_visibility,kp6_presence,kp7_x,kp7_y,kp7_z,kp7_visibility,kp7_presence,kp8_x,kp8_y,kp8_z,kp8_visibility,kp8_presence,kp9_x,kp9_y,kp9_z,kp9_visibility,kp9_presence,kp10_x,kp10_y,kp10_z,kp10_visibility,kp10_presence,kp11_x,kp11_y,kp11_z,kp11_visibility,kp11_presence,kp12_x,kp12_y,kp12_z,kp12_visibility,kp12_presence,kp13_x,kp13_y,kp13_z,kp13_visibility,kp13_presence,kp14_x,kp14_y,kp14_z,kp14_visibility,kp14_presence,kp15_x,kp15_y,kp15_z,kp15_visibility,kp15_presence,kp16_x,kp16_y,kp16_z,kp16_visibility,kp16_presence,kp17_x,kp17_y,kp17_z,kp17_visibility,kp17_presence,kp18_x,kp18_y,kp18_z,kp18_visibility,kp18_presence,kp19_x,kp19_y,kp19_z,kp19_visibility,kp19_presence,kp20_x,kp20_y,kp20_z,kp20_visibility,kp20_presence,kp21_x,kp21_y,kp21_z,kp21_visibility,kp21_presence,kp22_x,kp22_y,kp22_z,kp22_visibility,kp22_presence,kp23_x,kp23_y,kp23_z,kp23_visibility,kp23_presence,kp24_x,kp24_y,kp24_z,kp24_visibility,kp24_presence,kp25_x,kp25_y,kp25_z,kp25_visibility,kp25_presence,kp26_x,kp26_y,kp26_z,kp26_visibility,kp26_presence,kp27_x,kp27_y,kp27_z,kp27_visibility,kp27_presence,kp28_x,kp28_y,kp28_z,kp28_visibility,kp28_presence,kp29_x,kp29_y,kp29_z,kp29_visibility,kp29_presence,kp30_x,kp30_y,kp30_z,kp30_visibility,kp30_presence,kp31_x,kp31_y,kp31_z,kp31_visibility,kp31_presence,kp32_x,kp32_y,kp32_z,kp32_visibility,kp32_presence
0,0.504461,0.507432,0.514782,0.521345,0.494330,0.491313,0.487945,0.546749,0.502697,0.528674,0.512235,0.666130,0.512486,0.773310,0.399787,0.741734,0.252261,0.738761,0.208223,0.723615,0.207991,0.725950,0.226395,0.744547,0.642838,0.916164,0.423689,0.755264,0.226337,0.726184,0.203860,0.713343,0.176154,0.273989,0.234204,0.225700,0.218271,0.248725,0.252047,0.255582,0.202545,0.254539,0.290679,0.307171,0.232661,0.429213,0.083536,0.552453,0.273681,0.666265,0.322041,0.706553,0.326272,0.699131,0.312549,0.680602,0.714220,0.759108,0.511028,0.858214,0.521346,0.899377,0.522507,0.909439,0.484263,0.754127,-0.482882,-0.490882,-0.490885,-0.491087,-0.463354,-0.463471,-0.463413,-0.429502,-0.294632,-0.455665,-0.416732,-0.323857,-0.219179,-0.213966,-0.248411,-0.072627,-0.409637,-0.047386,-0.428687,-0.062467,-0.492485,-0.064804,-0.441154,-0.002963,0.002426,-0.177164,-0.134983,-0.167151,-0.471117,-0.155793,-0.499284,-0.183229,-0.585002,0.999986,0.999991,0.999978,0.999988,0.999992,0.999984,0.999987,0.999996,0.999972,0.999997,0.999995,0.999993,0.999978,0.996620,0.978975,0.943986,0.929945,0.875258,0.873812,0.862584,0.878303,0.859681,0.887926,0.999980,0.999969,0.985208,0.961928,0.863705,0.976946,0.911182,0.980135,0.827424,0.973536,1.000000,0.999999,0.999999,0.999999,0.999998,0.999998,0.999998,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,0.999979,0.999990,0.999960,0.999715,0.999861,0.999052,0.999899,0.998941,0.999934,0.999331,0.999997,0.999998,0.999454,0.999823,0.999827,0.999243,0.999744,0.998151,0.999569,0.997055
1,0.469645,0.465285,0.464494,0.463433,0.458680,0.453885,0.448294,0.438319,0.413166,0.459976,0.450295,0.465209,0.299603,0.624260,0.128489,0.791116,0.313107,0.833178,0.349877,0.839505,0.364437,0.826900,0.356486,0.276851,0.190881,0.560157,0.161380,0.854577,0.385158,0.893927,0.414597,0.911590,0.437986,0.549802,0.525639,0.524806,0.524042,0.524692,0.523727,0.522423,0.526426,0.522898,0.569347,0.567711,0.663602,0.589130,0.741552,0.540257,0.782153,0.563466,0.787822,0.572709,0.784454,0.571999,0.783986,0.568351,0.846753,0.822229,0.859918,0.596983,0.895007,0.613354,0.910877,0.614618,0.801264,0.586847,-0.499800,-0.469817,-0.469693,-0.469924,-0.480414,-0.480418,-0.480189,-0.288961,-0.330945,-0.431085,-0.442977,-0.159507,-0.300784,-0.275708,-0.455131,-0.501095,-0.671548,-0.535143,-0.721486,-0.589364,-0.692616,-0.532048,-0.663799,0.039794,-0.040478,-0

In [86]:
no_kp_files

['selected_poses/akarna/Akarna_Dhanurasana_0_18.jpg',
 'selected_poses/akarna/Akarna_Dhanurasana_25.jpg',
 'selected_poses/akarna/Akarna_Dhanurasana_0_377.jpg',
 'selected_poses/akarna/Akarna_Dhanurasana_0_54.jpg',
 'selected_poses/akarna/Akarna_Dhanurasana_12.jpg',
 'selected_poses/akarna/Akarna_Dhanurasana_0_4.jpg']